In [1]:
##################################
#
# Initial implementation of linear logic recurrent neural network
#
# The architecture is a modified RNN, see the paper "Linear logic and recurrent neural networks".
# Our inputs are sequences of symbols taken from an alphabet of size num_classes. The length
# of the sequences is N. Our outputs are also sequences of length N from the same alphabet.
#
# Here "symbol" means a one_hot vector.

# The next three lines are recommend by TF
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import collections
import six
import math
import time

from tensorflow.python.ops.rnn_cell_impl import _RNNCell as RNNCell
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh

# Our libraries
import ntm
import seqhelper
import learnfuncs

In [2]:
##############
# GLOBAL FLAGS

num_classes = 2
batch_size = 1000
input_size = num_classes # dimension of the input space I
N = 20 # length of sequences
training_percent = 0.01 # percentage used for training
epoch = 200

controller_state_size = 40 # dimension of the internal state space of the controller
memory_address_size = 20 # number of memory locations
memory_content_size = 5 # size of vector stored at a memory location
pattern_ntm_powers1 = [0,1,2,3]

use_model = 'pattern_ntm'

if( use_model == 'ntm' ):
    state_size = controller_state_size + 2*memory_address_size + memory_address_size * memory_content_size
elif( use_model == 'pattern_ntm'):
    state_size = controller_state_size + 4*memory_address_size + \
                memory_address_size * memory_content_size + \
                memory_address_size * len(pattern_ntm_powers1)

print("Total state size: " + str(state_size))

Total state size: 300


In [3]:
#######################
# PREPARE TRAINING DATA
#
# Our sequences are of one-hot vectors, which we interpret as follows:
#
# [1.0, 0.0] = 0
# [0.0, 1.0] = 1
#
# The sequences we input are binary, converted to one-hot vectors according to
# this dictionary.

# The function of sequences we are trying to approximate
pattern = [1,0,1,2,0,3,0,1,1,0,0,2,2,4,0,1,7,10,2,0,0]
func_to_learn = lambda s: learnfuncs.f_repetitionpattern(s,pattern)
#func_to_learn = learnfuncs.f_identity

# Create a shuffled list of all binary sequences of length N
seq_input = seqhelper.shuffled_binary_seqs(N)

one_hots = seqhelper.one_hot_vectors(num_classes)

seq_input_onehot = []
for i in seq_input:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_input_onehot.append(np.array(temp_list))

# Training output
seq_output = []

for i in seq_input:
    seq_output.append(func_to_learn(i))

seq_output_onehot = []
for i in seq_output:
    temp_list = []
    for j in i:
        temp_list.append(one_hots[j])
    seq_output_onehot.append(np.array(temp_list))

NUM_EXAMPLES = int(training_percent * len(seq_input))

test_input = seq_input_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
test_output = seq_output_onehot[NUM_EXAMPLES:3*NUM_EXAMPLES]
train_input = seq_input_onehot[:NUM_EXAMPLES]
train_output = seq_output_onehot[:NUM_EXAMPLES]

print("Number of training examples: " + str(NUM_EXAMPLES) + " out of " + str(len(seq_input)) + " sequences.")
print("")
print("Under the chosen function, the sequence")
print(seq_input[0])
print("which is encoded as")
print(seq_input_onehot[0])
print("is mapped to")
print(seq_output[0])
print("which is encoded as")
print(seq_output_onehot[0])

#print("")
#print("The first one-hot encoded digit of the first three output sequences")
#print(test_output[0][0])
#print(test_output[1][0])
#print(test_output[2][0])

Number of training examples: 10485 out of 1048576 sequences.

Under the chosen function, the sequence
[1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]
which is encoded as
[[ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]
is mapped to
[1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
which is encoded as
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]


In [4]:
# Definition of the model

# inputs, we create N of them, each of shape [None,input_size], one for
# each position in the sequence
inputs = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]
targets = [tf.placeholder(tf.float32, [None,input_size]) for _ in range(N)]

# We use tf.nn.rnn rather than dynamic_rnn because there appears to
# be a problem with tf.map_fn and the latter, at least in 0.10
# state_size is the number of hidden neurons in each layer

if( use_model == 'ntm' ):
    cell = ntm.NTM(state_size,input_size,controller_state_size,
                   memory_address_size,memory_content_size, [-1,0,1])
elif( use_model == 'pattern_ntm' ):
    cell = ntm.PatternNTM(state_size,input_size,controller_state_size,
                          memory_address_size,memory_content_size, pattern_ntm_powers1, [-1,0,1])

state = cell.zero_state(batch_size, tf.float32)

reuse = False

for i in range(N):
    output, state = cell(inputs[i],state,'NTM',reuse)
    reuse = True

# We only start recording the outputs of the controller once we have
# finished feeding in the input. We feed zeros as input in the second phase.
rnn_outputs = []
for i in range(N):
    output, state = cell(tf.zeros([batch_size,input_size]),state,'NTM',reuse)
    rnn_outputs.append(output)

# Final fully connected layer
E = tf.Variable(tf.truncated_normal([controller_state_size,input_size]))
F = tf.Variable(tf.constant(0.1, shape=[input_size]))

# prediction is a length N list of tensors of shape [None,input_size], where
# the jth row of prediction[d] is, for the jth input sequence in the batch,
# the probability distribution over symbols for the output symbol in position d.
logits = [tf.matmul(rnn_output, E) + F for rnn_output in rnn_outputs]
prediction = [tf.nn.softmax(logit) for logit in logits] 
ce = [tf.reduce_sum(targets[i] * tf.log(prediction[i])) for i in range(N)]

cross_entropy = -tf.add_n(ce)
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = [tf.not_equal(tf.argmax(targets[i], 1), tf.argmax(prediction[i], 1)) for i in range(N)]
errors = [tf.reduce_mean(tf.cast(m, tf.float32)) for m in mistakes]

[<tf.Tensor 'gradients/NTM_39/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_38/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_37/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_36/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_35/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_34/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_33/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_32/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_31/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_30/split_grad/concat:0' shape=(1000, 300) dtype=float32>, None, None]
[<tf.Tensor 'gradients/NTM_29/

In [5]:
# Initialise the model
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [6]:
# Display the errors before training
feed_dict = {}
test_input_batch = test_input[:batch_size]
test_output_batch = test_output[:batch_size]
for d in range(N):
    in_node = inputs[d]
    out_node = targets[d]
    
    ti = []
    to = []
    for k in range(len(test_input_batch)):
        ti.append(test_input_batch[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
        to.append(test_output_batch[k][d])
    feed_dict[in_node] = np.array(ti)
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
#print(sess.run(tf.argmax(targets[0],1),feed_dict))
#print(sess.run(tf.argmax(prediction[0],1),feed_dict))
#print(sess.run(tf.not_equal(tf.argmax(targets[0], 1), tf.argmax(prediction[0], 1)),feed_dict))

print("")
print("The mean of the errors in each digit for the test set:")
incorrects = sess.run(errors, feed_dict)
print(incorrects)
print("Mean: " + str(np.mean(incorrects)))


The mean of the errors in each digit for the test set:
[0.51899999, 0.49700001, 0.50400001, 0.53100002, 0.58099997, 0.491, 0.47299999, 0.63300002, 0.58499998, 0.59399998, 0.48500001, 0.38800001, 0.528, 0.65399998, 0.44400001, 0.47299999, 0.42899999, 0.46700001, 0.48300001, 0.46399999]
Mean: 0.51115


In [7]:
pre_train_time = time.time()

# Training
no_of_batches = int(len(train_input)/batch_size)
print("Number of batches: " + str(no_of_batches))

# An annoying thing here is that we cannot use a list as a key in a 
# dictionary. The workaround we found on StackOverflow here:
# http://stackoverflow.com/questions/33684657/issue-feeding-a-list-into-feed-dict-in-tensorflow)

# epoch is a global var
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp = train_input[ptr:ptr+batch_size]
        out = train_output[ptr:ptr+batch_size]
        ptr += batch_size
        
        feed_dict = {}
        for d in range(N):
            in_node = inputs[d]
            out_node = targets[d]
            
            # inp has dimensions [batch_size, N, num_classes] and we want to extract
            # the 2D Tensor of shape [batch_size, num_classes] obtained by setting the
            # second coordinate to d
            ti = []
            to = []
            for k in range(batch_size):
                ti.append(inp[k][d])
                to.append(out[k][d])

            feed_dict[in_node] = np.array(ti)
            feed_dict[out_node] = np.array(to)
            
        sess.run(minimize, feed_dict)
    print("Epoch - " + str(i+1) + ", Mean - " + str(np.mean(sess.run(errors, feed_dict))))
    
print("It took", time.time() - pre_train_time, "seconds to train.")

Number of batches: 10
Epoch - 1, Mean - 0.4934
Epoch - 2, Mean - 0.4657
Epoch - 3, Mean - 0.4589
Epoch - 4, Mean - 0.45605
Epoch - 5, Mean - 0.45185
Epoch - 6, Mean - 0.44465
Epoch - 7, Mean - 0.4332
Epoch - 8, Mean - 0.4227
Epoch - 9, Mean - 0.4115
Epoch - 10, Mean - 0.4055
Epoch - 11, Mean - 0.4004
Epoch - 12, Mean - 0.3966
Epoch - 13, Mean - 0.39385
Epoch - 14, Mean - 0.3834
Epoch - 15, Mean - 0.38055
Epoch - 16, Mean - 0.37425
Epoch - 17, Mean - 0.37305
Epoch - 18, Mean - 0.36405
Epoch - 19, Mean - 0.37125
Epoch - 20, Mean - 0.37805
Epoch - 21, Mean - 0.37425
Epoch - 22, Mean - 0.37285
Epoch - 23, Mean - 0.3704
Epoch - 24, Mean - 0.3664
Epoch - 25, Mean - 0.3608
Epoch - 26, Mean - 0.35465
Epoch - 27, Mean - 0.35065
Epoch - 28, Mean - 0.3589
Epoch - 29, Mean - 0.3512
Epoch - 30, Mean - 0.3518
Epoch - 31, Mean - 0.3483
Epoch - 32, Mean - 0.34775
Epoch - 33, Mean - 0.3516
Epoch - 34, Mean - 0.3463
Epoch - 35, Mean - 0.3454
Epoch - 36, Mean - 0.34295
Epoch - 37, Mean - 0.34455
Epoch - 

In [8]:
# Display the errors after training
feed_dict = {}
for d in range(N):
    in_node = inputs[d]
    out_node = targets[d]
    
    ti = []
    to = []
    for k in range(len(test_input_batch)):
        ti.append(test_input_batch[k][d]) # A vector giving the one-hot encoding of the dth symbol in the kth sequence
        to.append(test_output_batch[k][d])
    feed_dict[in_node] = np.array(ti)
    feed_dict[out_node] = np.array(to)

# The first three digits of this should match the printout for the
# first three test output sequences given earlier
data = sess.run([tf.argmax(targets[0],1),
                 tf.argmax(prediction[0],1)],feed_dict)

#print("First digits of test outputs (actual)")
#print(data[0])
#print("First digits of test outputs (predicted)")
#print(data[1])

# print the mean of the errors in each digit for the test set.
incorrects = sess.run(errors, feed_dict)
# print(incorrects)

print("############################")
print("# Summary ")
print("############################")
print("# model = " + use_model)
print("# training_percent = " + str(training_percent))
print("# epoch = " + str(epoch))
print("# (css,mas,mcs) = (" + str(controller_state_size) + "," + str(memory_address_size) + "," + str(memory_content_size) + ")")
print("# powers1 = " + str(pattern_ntm_powers1))
print("# number of weights = " + str(ntm.count_number_trainable_params()))
print("# error = " + str(np.mean(incorrects)))
sess.close()

############################
# Summary 
############################
# model = pattern_ntm
# training_percent = 0.01
# epoch = 200
# (css,mas,mcs) = (40,20,5)
# powers1 = [0, 1, 2, 3]
# number of weights = 3150
# error = 0.16435
